# Mineração de Dados para Inteligência de Negócio

## Semana 6: Workflows para Mineração de Dados

#### Importando as bibliotecas

In [242]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import OneClassSVM
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.datasets import load_breast_cancer

#### Carregando os dados do dataset breast_cancer

In [243]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X.shape, y.shape

((569, 30), (569,))

#### Exibindo os dados do dataset

In [244]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


#### Exibindo as classes

In [245]:
# diagnosis (0 = malignant, 1 = benign)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

#### Distribuindo os dados de treino e teste

In [246]:
SEED = 1234
np.random.seed(SEED)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=SEED, 
    shuffle=True,
    stratify=y
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((455, 30), (455,), (114, 30), (114,))

## Construindo a Pipeline com StandardScaler, RFECV e rede neural MLPClassifier

In [247]:
# StandardScaler
sc = StandardScaler()
# Recursive feature elimination with cross-validation
rfecv = RFECV(estimator=rfc, step=1, cv=5, scoring='accuracy')
# Multi-layer Perceptron classifier
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=SEED)

# Atribuindo os estimadores a pipeline
pipeline_mlp = make_pipeline(sc, rfecv, mlp)

#### Treinando o modelo

In [248]:
model_mlp = pipeline_mlp.fit(X_train, y_train)
model_mlp

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('rfecv',
                 RFECV(cv=5, estimator=RandomForestClassifier(),
                       scoring='accuracy')),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000,
                               random_state=1234))])

#### Avaliando o modelo

In [249]:
y_pred = model_mlp.predict(X_test)
print("Acurácia: ", (accuracy_score(y_test, y_pred) * 100).round(3))

Acurácia:  98.246


## Construindo a Pipeline com StandardScaler e detecção de novidades OneClassSVM

In [250]:
# Transforma os rótulos de classe em 1 e -1
y_train_transformed = y_train.copy()
y_train_transformed[y_train_transformed == 0] = -1
y_test_transformed = y_test.copy()
y_test_transformed[y_test_transformed == 0] = -1

# StandardScaler
sc = StandardScaler()
# One-Class SVM
oSVM = OneClassSVM(nu=0.25)

# Atribuindo os estimadores a pipeline
pipeline_oSVM = make_pipeline(sc, oSVM)

#### Treinando o modelo

In [251]:
model_oSVM = pipeline_oSVM.fit(X_train, y_train_transformed)
model_oSVM

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('oneclasssvm', OneClassSVM(nu=0.25))])

#### Avaliando o modelo

In [252]:
y_pred = model_oSVM.predict(X_test)
print("Acurácia: ", (accuracy_score(y_test_transformed, y_pred) * 100).round(3))

Acurácia:  71.053


In [253]:
mat = confusion_matrix(y_test_transformed, y_pred)
mat

array([[18, 24],
       [ 9, 63]], dtype=int64)

In [254]:
vn, fp, fn, vp = mat.ravel()

In [255]:
VPP = vp / (vp + fp)
VPP

0.7241379310344828

In [256]:
Sen = vp / (vp + fn)
Sen

0.875

In [257]:
VPN = vn / (vn + fn)
VPN

0.6666666666666666

In [258]:
Spe = vn / (vn + fp)
Spe

0.42857142857142855